In [5]:
from test import read_data

alibaba_edges = read_data('alibaba')

In [31]:
import bisect

class TemporalNode:
    def __init__(self):
        self.edges = []
        
    def add_edge(self, node, t):
        self.edges.append((node, t))
        
    def sort_edges(self):
        self.edges.sort(key=lambda x: x[1])
        
    def get_edges_greater_than_time(self, t):
        class TimestampKey:
            def __init__(self, time):
                self.time = time

            def __lt__(self, other):
                return self.time < other[1]
            
        index = bisect.bisect_right(self.edges, TimestampKey(t))
        return self.edges[index:]
            

class TemporalGraph:
    def __init__(self):
        self.nodes = {}
        
    def add_edge(self, u, i, t):
        if u not in self.nodes:
            self.nodes[u] = TemporalNode()    
        u_node = self.nodes[u]
        
        if i not in self.nodes:
            self.nodes[i] = TemporalNode()
        i_node = self.nodes[i]
        
        u_node.add_edge(i_node, t)
        
    def sort_edges(self):
        for node in self.nodes.values():
            node.sort_edges()
            
    def get_nodes(self):
        return list(self.nodes.values())
            
            
def get_all_temporal_paths(temporal_graph):
    def dfs(current_node, current_time, path, all_paths):
        edges = current_node.get_edges_greater_than_time(current_time)
        
        if not edges:
            all_paths.append(path)
            return
        
        for next_node, next_time in edges:
            new_path = path + [(next_node, next_time)]
            dfs(next_node, next_time, new_path, all_paths)
    
    all_paths = []
    
    for node in temporal_graph.get_nodes():
        dfs(node, -float('inf'), [], all_paths)
        print(f'Got paths for node {node}')
    
    return all_paths

In [6]:
import numpy as np

class TemporalNode:
    def __init__(self):
        # Use a list to store node connections and timestamps to reduce appending cost
        self.nodes = []  # Array of node IDs (connected nodes)
        self.timestamps = []  # Array of timestamps for each edge
        
    def add_edge(self, node_id, timestamp):
        # Append the new node ID and timestamp directly to the lists
        self.nodes.append(node_id)
        self.timestamps.append(timestamp)
        
    def sort_edges(self):
        # Convert lists to numpy arrays only when sorting
        self.nodes = np.array(self.nodes, dtype=int)
        self.timestamps = np.array(self.timestamps, dtype=int)
        
        # Sort the edges based on timestamps
        sorted_indices = np.argsort(self.timestamps)
        self.nodes = self.nodes[sorted_indices]
        self.timestamps = self.timestamps[sorted_indices]
        
    def get_edges_greater_than_time(self, t):
        # Return edges where the timestamp is greater than t using NumPy vectorized operations
        mask = np.array(self.timestamps) > t
        return self.nodes[mask], np.array(self.timestamps)[mask]
            

class TemporalGraph:
    def __init__(self):
        self.nodes = {}
        
    def add_edge(self, u, i, t):
        # Only create a new TemporalNode if it doesn't exist
        if u not in self.nodes:
            self.nodes[u] = TemporalNode()
        u_node = self.nodes[u]
        
        if i not in self.nodes:
            self.nodes[i] = TemporalNode()
        i_node = self.nodes[i]
        
        # Add the edge to both nodes (directed edges in the temporal graph)
        u_node.add_edge(i, t)
        i_node.add_edge(u, t)  # Add the reverse edge as well for undirected behavior (if required)
        
    def sort_edges(self):
        # Sort edges for all nodes
        for node in self.nodes.values():
            node.sort_edges()
            
    def get_nodes(self):
        return list(self.nodes.values())
            

def get_all_temporal_paths(temporal_graph, max_path_len=None):
    def dfs(current_node_id, current_time, path, all_paths, max_len):
        # Retrieve the TemporalNode for current_node_id
        current_node = temporal_graph.nodes[current_node_id]
        
        # Get edges greater than current_time using vectorized NumPy operations
        next_nodes, next_times = current_node.get_edges_greater_than_time(current_time)
        
        if len(next_nodes) == 0 or (max_path_len and len(path) >= max_len):
            # If no next nodes or maximum path length reached, append the path
            all_paths.append(path[:])  # Append a copy of the path when a leaf is reached
            return
        
        # Traverse the edges
        for next_node, next_time in zip(next_nodes, next_times):
            path.append((next_node, next_time))
            dfs(next_node, next_time, path, all_paths, max_len)
            path.pop()
    
    all_paths = []
    
    for node_id in temporal_graph.nodes:
        dfs(node_id, -float('inf'), [], all_paths, max_path_len)
        print(f'Got paths for node {node_id}')
    
    return all_paths

In [7]:
import numpy as np

class TemporalGraph:
    def __init__(self):
        # We will store nodes, edges, and timestamps as arrays
        self.nodes = []  # List of node IDs
        self.edges = []  # List of edges (pairs of node IDs)
        self.timestamps = []  # List of timestamps corresponding to edges
    
    def add_edge(self, u, i, t):
        # Append the edge and its timestamp to the lists
        self.nodes.append(u)
        self.nodes.append(i)
        self.edges.append((u, i))
        self.timestamps.append(t)
        
    def sort_edges(self):
        # Convert lists to NumPy arrays for efficient operations
        nodes = np.array(self.nodes)
        edges = np.array(self.edges)
        timestamps = np.array(self.timestamps)
        
        # Sort by timestamps
        sorted_indices = np.argsort(timestamps)
        
        # Reorder the edges and timestamps
        self.nodes = nodes[sorted_indices]
        self.edges = edges[sorted_indices]
        self.timestamps = timestamps[sorted_indices]
    
    def get_edges_for_node(self, node_id):
        # Return the indices of the edges that originate from node_id
        mask = self.nodes == node_id
        return self.edges[mask], self.timestamps[mask]

    def get_all_nodes(self):
        # Get unique nodes
        return np.unique(self.nodes)

def get_all_temporal_paths(temporal_graph, max_path_len=None):
    def dfs(current_node, current_time, path, all_paths, max_len):
        # Get edges for the current node that have a timestamp greater than current_time
        edges, timestamps = temporal_graph.get_edges_for_node(current_node)
        
        # Filter out the edges with timestamps less than or equal to current_time
        valid_edges = edges[timestamps > current_time]
        valid_timestamps = timestamps[timestamps > current_time]
        
        if len(valid_edges) == 0 or (max_path_len and len(path) >= max_len):
            # If no valid edges or maximum path length reached, append the path
            all_paths.append(path[:])  # Append a copy of the path when a leaf is reached
            return
        
        # Traverse the edges
        for (next_node, _), next_time in zip(valid_edges, valid_timestamps):
            path.append((next_node, next_time))
            dfs(next_node, next_time, path, all_paths, max_len)
            path.pop()
    
    all_paths = []
    
    # Get all unique nodes and perform DFS for each
    for node_id in temporal_graph.get_all_nodes():
        print(f'Getting paths for node {node_id}')
        dfs(node_id, -float('inf'), [], all_paths, max_path_len)
        print(f'Got paths for node {node_id}')
    
    return all_paths

In [8]:
temporal_graph = TemporalGraph()
for u, i, t in alibaba_edges:
    temporal_graph.add_edge(u, i, t)
    
print('Sorting edges')
temporal_graph.sort_edges()

Sorting edges


In [ ]:
all_paths = get_all_temporal_paths(temporal_graph, 100)

Getting paths for node 10


In [2]:
from temporal_walk import TemporalWalk

temporal_walk = TemporalWalk(
    num_walks=100_000,
    len_walk=50,
    picker_type='Linear',
    max_time_capacity=None
)

In [3]:
temporal_walk.add_multiple_edges(edges)

In [4]:
nx_graph = temporal_walk.to_networkx()